### Código para la P1

In [ ]:
using DifferentialEquations

function comovile_front!(du, u, p, t)
    mu, v, w = p
    du[1] = u[2]
    du[2] = (-w*sin(pi*u[1]) - v*u[2])/(1 - (v^2 / mu^2))
end

mu = 2.0
w = 1.0/pi
v = 2*sqrt(1/(1 + 4*w/(mu^2)))

u0 = [1 - 1e-3, 0.0]
p = [mu, v, w]
tspan = (0.0, 6000.0)
prob = ODEProblem(comovile_front!, u0, tspan, p)
sol = solve(prob, AutoVern7(Rodas5P()), saveat=0.01, abstol = 1e-10, reltol = 1e-10)
t = sol.t;

using GLMakie
fig = Figure()
ax1 = Axis(fig[1, 1], xlabel="t", ylabel="u(t), v(t)")
lines!(ax1, t, sol[1, :], label="u(t)")
lines!(ax1, t, sol[2, :], label="v(t)")
axislegend(ax1)

ax2 = Axis(fig[1, 2], xlabel="u(t)", ylabel="v(t)")
lines!(ax2, sol[1, :], sol[2, :])
display(fig)
tol = 1e-15
println("El valor mínimo alcanzado fue: ", minimum(sol[1, :]))
if minimum(sol[1, :]) < 0 - tol
    println("Está bajo la velocidad mínima.")
else
    println("Corresponde a un frente factible, o bien falta tiempo de integración.")
end

### Integración P3


In [1]:
using DifferentialEquations

function hyper_diffusion_front!(du, u, p, t)
    v, r, γ = p
    inv = 1/γ
    du[1] = u[2]
    du[2] = u[3]
    du[3] = u[4]
    du[4] = (r*u[1]*(1.0-u[1]) + v*u[2] + u[3])*inv
end

v = 2.0
r = 1
γ = 0.01
u0 = [0.001, 0.0, 0.0, 0.0]
p = [v, r, γ]
tspan = (10.0, 0.0)
prob = ODEProblem(hyper_diffusion_front!, u0, tspan, p)
sol = solve(prob, AutoTsit5(Rosenbrock23()), dtmax = 0.01)
t = sol.t;

using GLMakie

fig = Figure()
ax1 = Axis(fig[1, 1], xlabel="z", ylabel="u(z)")
lines!(ax1, t, sol[1, :], label="u(z)")
axislegend(ax1)

ax2 = Axis(fig[1, 2], xlabel="u(z)", ylabel="v(z)")
lines!(ax2, sol[1, :], sol[2, :])
display(fig)


InterruptException: InterruptException:

In [73]:
using DifferentialEquations

function hyper_diffusion!(du, u, p, t)
    r, γ, dx = p
    fourth_order_derivative = (u[5:end] .- 4*u[4:end-1] .+ 6*u[3:end-2] .- 4*u[2:end-3] .+ u[1:end-4])/dx^4
    second_order_derivative = (u[3:end] .- 2*u[2:end-1] .+ u[1:end-2])/dx^2
    du[3:end-2] = r .* u[3:end-2] .* (1.0 .- u[3:end-2]) .+ second_order_derivative[2:end-1] .- γ.* fourth_order_derivative
    du[2] = r .* u[2] .* (1.0 .- u[2]) .+ second_order_derivative[1] .- γ .* (u[4] - 4*u[3] + 6*u[2] - 4*u[1] + u[1])/dx^4
    du[1] = r .* u[1] .* (1.0 .- u[1]) .+ (u[2] .- 2*u[1] .+ u[1])/dx^2 .- γ .* (u[3] - 4*u[2] + 6*u[1] - 4*u[1] + u[1])/dx^4
    du[end-1] = r .* u[end-1] .* (1.0 .- u[end-1]) .+ second_order_derivative[end] .- γ .* (u[end-3] - 4*u[end-2] + 6*u[end-1] - 4*u[end] + u[end])/dx^4
    du[end] = r .* u[end] .* (1.0 .- u[end]) .+ (u[end] .- 2*u[end] .+ u[end-1])/dx^2 .- γ .* (u[end-2] - 4*u[end-1] + 6*u[end] - 4*u[end] + u[end])/dx^4
end

r = 1.0
γ = 1/8
dx = 0.5
x = 0:dx:200
u0 = zeros(length(x))#exp.((-x.^2)./1.0)
u0[1] = 1.0
p = [r, γ, dx]

tspan = (0.0, 110.0)
prob = ODEProblem(hyper_diffusion!, u0, tspan, p, progress=true)
sol = solve(prob, AutoTsit5(Rosenbrock23()), dtmax = 0.1)
t = sol.t;

┌ Warning: At t=66.9044051932279, dt was forced below floating point epsilon 1.4210854715202004e-14, and step error estimate = 1610.4494055466544. Aborting. There is either an error in your model specification or the true solution is unstable (or the true solution can not be represented in the precision of Float64).
└ @ SciMLBase /Users/miguel/.julia/packages/SciMLBase/sakPO/src/integrator_interface.jl:633


In [74]:
using GLMakie
using Printf

fig = Figure()
ax1 = Axis(fig[1, 1], xlabel="x", ylabel="u(t_i, x)")
limits!(ax1, 0, 200, -0.1, 1.1)
n_time_intervals=10
times = range(0, stop=tspan[2], length=n_time_intervals)
for time in times
    index = argmin(abs.(sol.t .- time))
    lines!(ax1, x, sol.u[index], label="t = $(@sprintf("%.1f", time))")
end
axislegend(ax1)
display(fig)


GLMakie.Screen(...)

In [70]:
using GLMakie
speed = (2/3) * sqrt((1 - sqrt(1- 12*γ*r))/(6*γ))*(2 - sqrt(1 - 12*γ*r))
println("La velocidad de la onda es: ", speed)

fig = Figure()
ax1 = Axis(fig[1, 1], xlabel="x", ylabel="t")
heatmap!(ax1, x, sol.t, sol[1:end, 1:end], colormap = :RdGy)
lines!(ax1, x, x./speed, color = :green, linewidth = 6)
display(fig)


La velocidad de la onda es: 1.8856180831641267


GLMakie.Screen(...)

In [55]:
speed = r/sqrt(1/(6*γ) + sqrt(r + (1 / (6*γ))^2)) + sqrt(1/(6*γ) + sqrt(r + (1 / (6*γ))^2)) + γ * sqrt(1/(6*γ) + sqrt(r + (1 / (6*γ))^2))^(3/2)


2.790099256446529